# Ansible


Ansible is a configuration management software.

Connects via ssh or docker to a list of inventory machines and executes a series of tasks eventually grouped in playbooks.

At first, create an inventory file with all your hosts.

In [1]:
cd /notebooks/exercise-00  

/notebooks/exercise-00


In [2]:
cat inventory

#
# This inventory file contains a list of server to
#  play with - divided in groups.
#
[course]
# this is the local machine where you run jupyter
# the tutorial just works with this one.
pythonforsysadmin_course_1  ansible_connection=local


# Another group of servers
# where we can pass optional arguments
# Homework: you can play with this group of host
#  once you exchange ssh-keys between the pythonforsysadmin_course_1
#  container and the pythonforsysadmin_ansible_* ones._
[ansible]
172.17.0.[5:7]

#
# Besides, ansible has two predefined groups:
#   - all
#   - ungrouped


Now check if you can ping the local host.

In [3]:
# Check connections versus the local host in the "course" group

!ansible -i inventory -m ping course

pythonforsysadmin_course_1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


In [4]:
# Pinging all hosts gives some errors too, due to missing hosts or no ssh-key exchange
!ansible -i inventory -m ping all

pythonforsysadmin_course_1 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
172.17.0.5 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.5 port 22: No route to host\r\n", 
    "unreachable": true
}
172.17.0.7 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.7 port 22: No route to host\r\n", 
    "unreachable": true
}
172.17.0.6 | UNREACHABLE! => {
    "changed": false, 
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 172.17.0.6 port 22: No route to host\r\n", 
    "unreachable": true
}


## Further on inventories

 You can split your servers in many inventory files, like

 - staging
 
```
# staging inventory file
# run with
# ansible -i staging ...
 [ws]
 staging-ws-[0:3]
 
 [jboss]
 staging-boss-[0:6]
```
 
 - production
 
```
# production inventory file
# run with
# ansible -i production ...
 [ws]
 ws-[0:3]
 
 [jboss]
 boss-[0:6]
```
 
 

# Playbooks

To run a group of tasks with ansible, just:

  - create a playbook.yml
  - run ansible-playbook -i inventory playbook.yml
  

A playbook is a list of tasks in yml format, something like

```
#
# playbook.yml
#
- name: >-
    All public traffic is redirected via https.
    Beware: in real world, if your site accept credentials,
            you should close port 80 instead!
  uri:
    url: http://{{server_host}}/
    follow_redirects: none
    status_code: 301
    
- name: This webapp  is served
  uri:
    url: https://{{server_host}}/webapp-1
    validate_certs: false
    status_code: 200
    HEADER_testflag: test

- name: The WS is serverd and requires authentication
  uri:
    url: https://{{server_host}}/rest/v1/method
    validate_certs: false
    status_code: 401

```

In this case, instead of making actual installation|setup tasks, we just created a testsuite validating our deployment. Now we must write another playbook which takes care of deployng the actual machine.



## Testing our course environment

We can write a playbook to test our course environment.

In [5]:
!cat python-course-test.yml

# Run this with
#
#   #ansible-playbook -i inventory python-course-test.yml
#
- hosts: course
  tasks:
    - name: The /notebooks directory should exist
      file: path="/notebooks" state=directory

    - name: jupyter is responding on 8888
      uri:
        url: http://0.0.0.0:8888/notebooks
        validate_certs: false
        status_code: 200  # modify this line to simulate an error and see the outcome!

    - name: The template.conf is in place
      file: path="/tmp/template.conf" state=file


In [6]:
!ansible-playbook -i inventory python-course-test.yml 


PLAY [course] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [pythonforsysadmin_course_1]

TASK [The /notebooks directory should exist] ***********************************
ok: [pythonforsysadmin_course_1]

TASK [jupyter is responding on 8888] *******************************************
ok: [pythonforsysadmin_course_1]

TASK [The template.conf is in place] *******************************************
fatal: [pythonforsysadmin_course_1]: FAILED! => {"changed": false, "msg": "file (/tmp/template.conf) is absent, cannot continue", "path": "/tmp/template.conf", "state": "absent"}

PLAY RECAP *********************************************************************
pythonforsysadmin_course_1 : ok=3    changed=0    unreachable=0    failed=1   



As you can see something is missing: this playbook is not going to modify our machine but only test that everything is in place. 

See ```ansible-playbook --check ``` and ``` --diff ``` for further infos.

We can run a setup playbook, conventionally named [site.yml (click to edit)](/edit/notebooks/exercise-00/site.yml).


In [7]:
!cat site.yml

# Run this with
#
#   #ansible-playbook -i inventory site.yml
#
- hosts: course
  tasks:

    - name: Create a file from a template
      template: src=mytemplate.j2 dest=/tmp/template.conf

    - name: Ensure needed packages are present and eventually install them
      apt: name={{item}} state=present
      with_items:
        - python
        - python-dev
      ignore_errors: yes  #  ignore errors during the course (eg. connectivity)


In [8]:
!ansible-playbook -i inventory site.yml --limit=course  # in this case the --limit does not change anything ;)


PLAY [course] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [pythonforsysadmin_course_1]

TASK [Create a file from a template] *******************************************
changed: [pythonforsysadmin_course_1]

TASK [Ensure needed packages are present and eventually install them] **********
failed: [pythonforsysadmin_course_1] (item=[u'python', u'python-dev']) => {"changed": false, "cmd": "apt-get update", "item": ["python", "python-dev"], "msg": "W: Failed to fetch http://security.debian.org/dists/jessie/updates/main/binary-amd64/Packages  404  Not Found [IP: 151.101.130.132 80]\n\nW: Failed to fetch http://deb.debian.org/debian/dists/jessie/main/binary-amd64/Packages  404  Not Found\n\nW: Failed to fetch http://deb.debian.org/debian/dists/jessie-updates/main/binary-amd64/Packages  404  Not Found\n\nE: Some index files failed to download. They have been ignored, or old ones used 

In [10]:
!ansible --version

ansible 2.4.2.0
  config file = /notebooks/exercise-00/ansible.cfg
  configured module search path = [u'/root/.ansible/plugins/modules', u'/usr/share/ansible/plugins/modules']
  ansible python module location = /usr/local/lib/python2.7/site-packages/ansible
  executable location = /usr/local/bin/ansible
  python version = 2.7.14 (default, Dec 12 2017, 16:55:09) [GCC 4.9.2]
